In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain_ollama import ChatOllama

model_name="granite4:1b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url,
)

## Semantic search

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("resources/acmecorp-employee-handbook.pdf")

data = loader.load()

print(data)

incorrect startxref pointer(1)
parsing for Object Streams


[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-11-20T23:23:16+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-11-20T23:23:16+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resources/acmecorp-employee-handbook.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Employee Handbook\nNon-Disclosure Agreement (NDA) Policy\nEmployees must protect confidential information belonging to the company, its clients, and partners.\nThis includes, but is not limited to, product roadmaps, customer data, internal communications,\nproprietary algorithms, financial information, and unreleased features. Confidential information may not\nbe shared with unauthorized individuals inside or outside the organization. These obligations continue\nafter employment ends.\nWorkplace Conduct Policy\nEmployees must maintain a respectful, professional environment

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

3


Embedding Models: https://docs.langchain.com/oss/python/integrations/text_embedding

In [10]:
#from langchain_openai import OpenAIEmbeddings
#embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings( 
    model="nomic-embed-text:latest",
    base_url=model_url,
)

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [12]:
ids = vector_store.add_documents(documents=all_splits)

In [13]:
results = vector_store.similarity_search(
    "How many days of vacation does an employee get in their first year?"
)

print(results[0])

page_content='prohibited. Violations may result in disciplinary action.
Paid Time Off (PTO) Policy
Full■time employees accrue PTO according to the following schedule:  0–1 years of service: 10 days
per year (0.833 days per month)  1–3 years of service: 15 days per year (1.25 days per month)  3+
years of service: 20 days per year (1.67 days per month) PTO may be used for vacation, personal
needs, or illness. Requests should be submitted in advance through the HR system unless related to
an emergency. Employees may carry over up to 5 unused PTO days per calendar year. Extended
absences exceeding 5 consecutive business days require manager approval.
Travel & Expense Policy
Employees may be reimbursed for reasonable and necessary expenses incurred during approved
business travel. This includes transportation, lodging, meals, and incidental expenses within
established limits. Receipts must be submitted within 14 days of travel. First-class travel, personal' metadata={'producer': 'ReportL

## RAG Agent

In [9]:
from langchain.tools import tool

@tool
def search_handbook(query: str) -> str:
    """Search the employee handbook for information"""
    results = vector_store.similarity_search(query)
    return results[0].page_content

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[search_handbook],
    system_prompt="You are a helpful agent that can search the employee handbook for information."
    )

In [11]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many days of vacation does an employee get in their first year?")]}
)

In [12]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='How many days of vacation does an employee get in their first year?', additional_kwargs={}, response_metadata={}, id='42b8c788-d492-49c4-a9d1-08b6cfe1fb27'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-27T00:14:59.482131Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6604719700, 'load_duration': 4666942300, 'prompt_eval_count': 194, 'prompt_eval_duration': 457893200, 'eval_count': 34, 'eval_duration': 1349519300, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b5d28-73e0-7582-881a-248a9503c43a-0', tool_calls=[{'name': 'search_handbook', 'args': {'query': 'How many days of vacation does an employee get in their first year?'}, 'id': '5e9df2fc-7e40-437e-ba8d-500f0ceb887c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 194, 'output_tokens': 34, 'total_tokens': 228}),
              ToolMessage(content='prohibited. 

In [13]:
print(response["messages"][-1].content)

Based on the PTO Policy section in the employee handbook, full-time employees accrue Paid Time Off (PTO) according to their years of service:

- **0–1 years of service:** 10 days per year (approximately 0.833 days per month)
- **1–3 years of service:** 15 days per year (approximately 1.25 days per month)
- **3+ years of service:** 20 days per year (approximately 1.67 days per month)

This policy allows employees to use their PTO for vacation, personal needs, or illness. Requests should be submitted in advance through the HR system unless related to an emergency. Employees may carry over up to 5 unused PTO days per calendar year.
